In [1]:
import importlib

import model
import pan_loader
import base_config
import loss_functions as L

import utils

import torch
import torch.nn as nn
import torch.optim as optim

import os
import time
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [2]:
data_dir = "/home/aravind/dataset/"
ann_dir = data_dir + "annotations/panoptic/"

train_img_dir = data_dir + "train2017/"
train_seg_dir = ann_dir + "panoptic_train2017/"
train_ann_json = ann_dir + "panoptic_train2017.json"

val_img_dir = data_dir + "val2017/"
val_seg_dir = ann_dir + "panoptic_val2017/"
val_ann_json = ann_dir + "panoptic_val2017.json"

# train_img_dir = val_img_dir 
# train_seg_dir = val_seg_dir 
# train_ann_json = val_ann_json 

In [3]:
with open(val_ann_json,"r") as f:
    val_ann = json.load(f)
with open(train_ann_json,"r") as f:
    train_ann = json.load(f)

In [4]:
config = base_config.Config()

In [5]:
train_loader = pan_loader.get_loader(train_img_dir, train_seg_dir, train_ann, config)
val_loader = pan_loader.get_loader(val_img_dir, val_seg_dir, val_ann, config)

In [6]:
net = model.hgmodel()
# model_dir="models/"
# model_name="first_0.pt"
# pretrained_dict = torch.load(model_dir+model_name)
# net_dict = net.state_dict()

# pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in net_dict}
# net_dict.update(pretrained_dict) 
# net.load_state_dict(net_dict)


In [7]:
def set_trainable(module,state):
    for param in module.parameters():
        param.requires_grad = state

set_trainable(net,False)

set_trainable(net.mb0,True)
# set_trainable(net.mb1,True)
set_trainable(net.cb,True)

set_trainable(net.iresnet0, False)
# set_trainable(net.iresnet1, False)

for name,module in net.iresnet0.named_modules():
    if 'copy_bn' in name:
        set_trainable(module, False)
    elif 'copy_conv' in name:
        set_trainable(module, False)
    elif 'wing' in name:
        set_trainable(module, True)
    else:
        set_trainable(module, False)
        if isinstance(module, nn.BatchNorm2d):
            module.eval()
        
param_lr = []
param_lr.append({'params': net.mb0.parameters(),'lr':1e-4,'momentum':0.9})
# param_lr.append({'params': net.mb1.parameters(),'lr':1e-4,'momentum':0.9})
param_lr.append({'params': net.cb.parameters(),'lr':1e-4,'momentum':0.9})

# for name,module in net.iresnet0.named_modules():
#     if 'copy_bn' in name:
#         param_lr.append({'params':module.parameters(),'lr':1e-4,'momentum':0.9})
#     elif 'copy_conv' in name:
#         param_lr.append({'params':module.parameters(),'lr':1e-4,'momentum':0.9})

for name,child in net.iresnet0.named_children():
    if 'wing' in name:
        param_lr.append({'params':child.parameters(),'lr':1e-4,'momentum':0.9})

# param_lr = [{'params':net.parameters(),'lr':1e-4, 'momentum':0.9}]
net_size = sum([i.numel() for i in net.parameters()])
trainable_params = filter(lambda p: p.requires_grad, net.parameters())
trainable_size = sum([i.numel() for i in trainable_params])
print(net_size,trainable_size)
optimizer = optim.SGD(param_lr, weight_decay=1e-4)

37761792 11222096


In [ ]:
# net = nn.DataParallel(net, device_ids=[0,1])
net = net.cuda()

In [ ]:
ckpt = utils.Checkpoint(iters_per_epoch=20, model_dir="./models/", model_name="fourth")
for i, data in enumerate(train_loader,0):
    optimizer.zero_grad()
    
    images, impulses, instance_masks, cat_ids = utils.cudify_data(data)
    mask_logits, cat_scores = net([images,instance_masks])
    mask_loss = L.balanced_bce(mask_logits, instance_masks.unsqueeze(1))

    loss = mask_loss
    mean_iou = L.mean_iou(mask_logits, instance_masks.unsqueeze(1))
    ckpt.update({"mask_loss":mask_loss, "mean_iou":mean_iou}, net)
    loss.backward()
    optimizer.step()